<a href="https://colab.research.google.com/github/Saikiran156/Git-pack/blob/master/language_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import re
import numpy as np
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, RepeatVector, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
pd.set_option('display.max_colwidth', 200)

THIS DATA TEXT FILE CONSISTS OF GERMAN-ENGLISH SENTENCE PAIRS .NOW WE HAVE TO READ THE DATA

In [ ]:
#FUNCTION TO READ THE RAW TEXT FILE
def read_text(filename):
  #opening the file
  file = open(filename, mode='rt', encoding='utf-8')
  #read all text
  text = file.read()
  #closing the file
  file.close()
  return text

IT'S TIME TO SPLIT THE PLAIN TEXT INTO ENGLISH-GERMAN PAIRS SEPERATED BY '\n' AND THEN SPLIT THESE PAIRS INTO ENGLISH AND GERMAN SENTENCES.

In [ ]:
#split a text into sentences
def to_lines(text):
  sents = text.strip().split('\n')
  sents = [i.split('\t') for i in sents]
  return sents

In [ ]:
data = read_text("/content/deu.txt")
deu_eng = to_lines(data)
deu_eng = array(deu_eng)

THE ACTUAL DATASET IS VERY HUGE.THEREFORE WE USE FIRST 50000 SENTENCE PAIRS FOR REDUCING THE TRAINING TIME OF THE MODEL 

In [ ]:
deu_eng = deu_eng[:500,:]

In [ ]:
#this is our data
deu_eng

TEXT TO SEQUENCE CONVERSION 
TO FEED OUR DATA IN A Seq2Seq MODEL, WE WILL HAVE TO CONVERT BOTH INPUT AND THE OUTPUT SENTENCES INTO INTEGER SEQUENCES OF FIXED LENGTH. BEFOR THAT, LET'S VISUALIZE THE LENGTH OF THE SENTENCES. WE WILL CAPTURE THE LENGTHS OF ALL THE SENTENCES IN TWO SEPERATE LISTS FOR ENGLISH AND GERMAN, RESPECTIVELY

In [ ]:
#empty lists
eng_1 = []
deu_1 = []

#populate the lists with sentence lengths
for i in deu_eng[:,0]:
  eng_1.append(len(i.split()))
for i in deu_eng[:,1]:
  deu_1.append(len(i.split()))

In [ ]:
length_df = pd.DataFrame({'eng':eng_1, 'deu':deu_1})

In [ ]:
length_df.hist(bins = 30)
plt.show()

THE MAXIMUM LENGTH OF GERMAN SENTENCES IS 9 AND THAT OF THE ENGLISH PHRASES IS 8.
LET'S VECTORIZE OUR TEXT DATA BY USING KERAS'S TOKENIZER() CLASS. IT WILL TURN OUR SENTENCES INTO SEQUENCES OF INTEGERS. THEN WE WILL PAD THOSE SEQUENCES WITH ZEROS TO MAKE ALL THE SENTENCES OF SAME LENGTH.

In [ ]:
#funtion to build a tokenizer
def tokenization(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer

In [ ]:
#prepare english tokenizer
eng_tokenizer = tokenization(deu_eng[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index)+1

eng_length = 8
print('English Vocabulary Size: %d' % eng_vocab_size)

English Vocabulary Size: 217


In [ ]:
# prepare Deutch tokenizer
deu_tokenizer = tokenization(deu_eng[:, 1])
deu_vocab_size = len(deu_tokenizer.word_index) + 1

deu_length = 8
print('Deutch Vocabulary Size: %d' % deu_vocab_size)

Deutch Vocabulary Size: 400


In [ ]:
#HERE IS THE FUNCTION TO PREPARE THE SEQUENCES.IT WILL ALSO PERFORM SEQUENCE PADDING TO A MAXIMUM SENTENCE LENGTH AS MENTIONED ABOVE

In [ ]:
#encode and pad sequences
def encode_sequences(tokenizer,length,lines):
  #integer encode sequences
  seq = tokenizer.texts_to_sequences(lines)
  #pad sequences with 0 values
  seq = pad_sequences(seq,maxlen=length,padding='post')
  return seq

MODEL BUILDING
WE WILL NOW SPLIT THE DATA INTO TRAIN AND TEST SET FOR MODEL TRAINING AND EVALUATION,RESPECTIVELY

In [ ]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(deu_eng,test_size=0.2,random_state = 12)

IT'S TIME TO ENCODE THE SENTENCES. WE WILL ENCODE GERMAN SENTENCES AS THE INPUT SEQUENCES AND ENGLISH SENTENCES AS THE TARGET SEQUENCES.IT WILL BE DONE FOR BOTH TRAIN AND TEST DATASETS.

In [ ]:
# preparing training data
trainX = encode_sequences(deu_tokenizer, deu_length, test[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])

In [ ]:
# preparing validation data
testX = encode_sequences(deu_tokenizer, deu_length,test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length,test[:, 0])

NOW COMES THE EXICITING PART! LET US DEFINE OUR Seq2Seq MODEL ARCHITECTURE.WE ARE USING AN EMBEDDING LAYER AND AN LSTM LAYER AS OUR ENCODER AND ANOTHER LSTM LAYER FOLLOWED BY A DENSE LAYER AS THE DECODER

In [ ]:
# BUILD NMT MODEL
def build_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
  model = Sequential()
  model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
  model.add(LSTM(units))
  model.add(RepeatVector(out_timesteps))
  model.add(LSTM(units, return_sequences=True))
  model.add(Dense(out_vocab, activation='softmax'))
  return model

In [ ]:
model = build_model(deu_vocab_size, eng_vocab_size, deu_length, eng_length, 512)
ms = tf.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=ms, loss='sparse_categorical_crossentropy')

WE ARE USING RMSprop OPTIMIZER IN THIS MODEL AS IT IS USUALLY A GOOD CHOICE FOR RECURRENT NEURAL NETWORKS

IT'S TIME TO START TRAINING THE MODEL.NOW TRAIN IT FOR 30 EPOCHS AND WITH A BATCH SIZE OF 512.NOW WE WILL ALSO BE USING ModelCheckpoint() to 
save the best model with lowest validation loss

In [ ]:
filename = 'model.h1.24_aashutosh'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=5, batch_size=512,
                    validation_split = 0.2,
                    callbacks=[checkpoint], verbose=1)


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

In [ ]:
model = load_model('model.h1.24_aashutosh')
preds = model.predict(testX.reshape((testX.shape[0],testX.shape[1])))
preds = np.argwhere(preds)

In [ ]:
def get_word(n, tokenizer):
  for word,index in tokenizer.word_index.items():
    if index == n:
      return word
  return None

In [ ]:
#convert predicitons into text (ENGLISH)
preds_text = []
for i in preds:
  temp = []
  for j in range(len(i)):
    t = get_word(i[j], eng_tokenizer)
    if j>0:
       if(t == get_word(i[j-1], eng_tokenizer)) or (t == None):
          temp.append('')
       else:
          temp.append(t)
    
    else:
        if(t == None):
            temp.append('')
        else:
            temp.append(t)

  preds_text.append(''.join(temp))
  
  

In [ ]:
pred_df = pd.DataFrame({'actual' : test[:,0]})
preds_text = array(preds_text)
a = pd.DataFrame({'predicted' : preds_text})

In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
frames = [pred_df,a]
result = pd.concat(frames,axis=1)
result.head(20)

,actual,predicted
0,I'm full.,
1,Get lost!,i
2,Go away!,it
3,Ask me.,away
4,Find Tom.,i'm
5,Ask me.,get
6,It's me.,go
7,I'm deaf.,tom
8,Wow!,me
9,Go away!,lost
